### 1. Generating dataset:

In [1]:
import os
import sys
from pathlib import Path
import uuid
from tqdm import tqdm
from typing import List
import pandas as pd
import re
import json
from dotenv import load_dotenv
sys.path.append(str(Path.cwd().parent.parent))

from llama_index.core import Document 
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI
from llama_index.core.schema import MetadataMode, TextNode
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core.evaluation import (
    generate_question_context_pairs,
    EmbeddingQAFinetuneDataset,
)
from llama_index.core.llama_dataset.legacy.embedding import generate_qa_embedding_pairs


from qdrant_client import QdrantClient

load_dotenv(override=True)

True

In [3]:
# Init Êmbedding Model:
embed_model = OpenAIEmbedding(model='text-embedding-3-large', api_key=os.getenv('OPENAI_API_KEY'))
Settings.embed_model = embed_model
llm = OpenAI(model='gpt-4o-mini', api_key=os.getenv('OPENAI_API_KEY'))
Settings.llm = llm

In [4]:
# get all node from qdrant database:
def get_nodes_from_collection(collection_name: str):
    '''
    Get all nodes from a qdrant collection
    Args:
        collection_name: a qdrant collection name
    '''
    client = QdrantClient(url="http://localhost:6333")

    qdrant_nodes, _ = client.scroll(
        collection_name="contextual_rag_nckh",
        limit=1489
    )
    nodes = []
    for node in qdrant_nodes:
        nodes.append(TextNode(text=node.payload['text'], id_=node.id))
    return nodes

nodes = get_nodes_from_collection('contextual_rag_nckh')

/tmp/ipykernel_3495452/3645926139.py:8: UserWarning: Qdrant client version 1.13.3 is incompatible with server version 1.11.3. Major versions should match and minor version difference must not exceed 1. Set check_version=False to skip version check.
  client = QdrantClient(url="http://localhost:6333")


In [5]:
qa_dataset = generate_qa_embedding_pairs(
    llm = Settings.llm,
    nodes = nodes, 
    #qa_generate_prompt_tmpl = DEFAULT_CONTEXT_QUESTION_PAIR_PROMPT,
    num_questions_per_chunk = 2
)

100%|██████████| 1489/1489 [49:50<00:00,  2.01s/it]


#### 1.1. Save question context pairs to json:

In [7]:
def custom_save_json(dataset: EmbeddingQAFinetuneDataset, path: str = './qa_dataset.json'):
    queries = dataset.queries
    corpus = dataset.corpus
    relevant_docs = dataset.relevant_docs

    data = {
        "queries": queries,
        "corpus": corpus,
        "relevant_docs": relevant_docs
    }

    with open(path, 'w', encoding="utf-8") as f:
        json.dump(data, f, indent=4, ensure_ascii=False)
    
custom_save_json(qa_dataset, )

In [8]:
qa_dataset.save_json('/workspace/competitions/Sly/RAG_Traffic_Law_experiment/duy/notebook/evaluate/source/qa_test.json')

### 2. Retrieval Pipeline Building:


In [5]:
from llama_index.vector_stores.qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client import AsyncQdrantClient, models


from llama_index.llms.openai import OpenAI
from llama_index.postprocessor.rankgpt_rerank import RankGPTRerank
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core import (
    Settings,
    Document,
    QueryBundle,
    StorageContext,
    VectorStoreIndex,
)

import nest_asyncio
nest_asyncio.apply()


In [6]:
# Dùng async client
async_client = AsyncQdrantClient(url="http://localhost:6333")
qdrant_client = QdrantClient(url="http://localhost:6333")
vector_store = QdrantVectorStore(aclient=async_client, client=qdrant_client, collection_name="contextual_rag_nckh")
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# Phải bật use_async
qdrant_index = VectorStoreIndex.from_vector_store(
    vector_store=vector_store, 
    storage_context=storage_context,
    use_async=True
)

retriever = qdrant_index.as_retriever(similarity_top_k=10)

/tmp/ipykernel_3495452/3703757450.py:2: UserWarning: Qdrant client version 1.13.3 is incompatible with server version 1.11.3. Major versions should match and minor version difference must not exceed 1. Set check_version=False to skip version check.
  async_client = AsyncQdrantClient(url="http://localhost:6333")
/tmp/ipykernel_3495452/3703757450.py:3: UserWarning: Qdrant client version 1.13.3 is incompatible with server version 1.11.3. Major versions should match and minor version difference must not exceed 1. Set check_version=False to skip version check.
  qdrant_client = QdrantClient(url="http://localhost:6333")
Both client and aclient are provided. If using `:memory:` mode, the data between clients is not synced.


In [7]:
# test:
retrieved_nodes = retriever.retrieve("Tôi vượt đèn đỏ xử lí thế nào?")

KeyboardInterrupt: 

In [80]:
retrieved_nodes

[NodeWithScore(node=TextNode(id_='9bcd3a5f-1f7f-4cb1-9961-d8ade0b46e6b', embedding=None, metadata={'chapter_uuid': '', 'original_content': 'Tín hiệu đèn giao thông có 03 màu, gồm: màu xanh, màu vàng, màu đỏ; có hiển thị thời gian hoặc không hiển thị thời gian. Người tham gia giao thông đường bộ phải chấp hành như sau: a) Tín hiệu đèn màu xanh là được đi; trường hợp người đi bộ, xe lăn của người khuyết tật đang đi ở lòng đường, người điều khiển phương tiện tham gia giao thông đường bộ phải giảm tốc độ hoặc dừng lại nhường đường cho người đi bộ, xe lăn của người khuyết tật qua đường; b) Tín hiệu đèn màu vàng phải dừng lại trước vạch dừng; trường hợp đang đi trên vạch dừng hoặc đã đi qua vạch dừng mà tín hiệu đèn màu vàng thì được đi tiếp; trường hợp tín hiệu đèn màu vàng nhấp nháy, người điều khiển phương tiện tham gia giao thông đường bộ được đi nhưng phải quan sát, giảm tốc độ hoặc dừng lại nhường đường cho người đi bộ, xe lăn của người khuyết tật qua đường hoặc các phương tiện khác; c

### 3. Evaluating Retrieval:

In [ ]:
from llama_index.core.evaluation import RetrieverEvaluator

In [ ]:
qa_dataset = EmbeddingQAFinetuneDataset.from_json("./qa_dataset.json")
metrics = ["hit_rate", "mrr"]

reranker = RankGPTRerank(
            llm = Settings.llm,
            top_n=10,
            verbose=True,
)


retriever_evaluator = RetrieverEvaluator.from_metric_names(
    metric_names=metrics,
    retriever=retriever,
    node_postprocessors=[reranker]
)

In [14]:
# try it out on a sample query
sample_id, sample_query = list(qa_dataset.queries.items())[0]
sample_expected = qa_dataset.relevant_docs[sample_id]

eval_result = retriever_evaluator.evaluate(sample_query, sample_expected)
print(eval_result)

After Reranking, new rank list for nodes: [3, 2, 9, 1, 0, 4, 5, 6, 7, 8]Query: Theo Điều 32 trong Nghị định 165/2024/NĐ-CP, chủ đầu tư có những trách nhiệm gì trước khi đưa công trình đường bộ vào khai thác?
Metrics: {'hit_rate': 1.0, 'mrr': 1.0}



In [ ]:
# try it out on an entire dataset
eval_results = await retriever_evaluator.aevaluate_dataset(qa_dataset, workers=50)

After Reranking, new rank list for nodes: [2, 0, 1, 4, 5, 3, 6, 9, 8, 7]After Reranking, new rank list for nodes: [1, 4, 0, 9, 2, 3, 8, 5, 7, 6]After Reranking, new rank list for nodes: [4, 1, 9, 0, 3, 2, 6, 7, 5, 8]After Reranking, new rank list for nodes: [1, 0, 2, 3, 4, 5, 6, 7, 8, 9]After Reranking, new rank list for nodes: [1, 2, 0, 7, 8, 4, 5, 9, 3, 6]After Reranking, new rank list for nodes: [1, 3, 0, 9, 2, 4, 6, 8, 5, 7]After Reranking, new rank list for nodes: [1, 9, 2, 6, 0, 3, 4, 5, 7, 8]After Reranking, new rank list for nodes: [0, 3, 2, 8, 4, 5, 1, 9, 7, 6]After Reranking, new rank list for nodes: [0, 2, 4, 9, 1, 5, 7, 8, 3, 6]After Reranking, new rank list for nodes: [2, 0, 1, 9, 7, 6, 5, 3, 4, 8]After Reranking, new rank list for nodes: [2, 5, 0, 1, 3, 4, 8, 6, 7, 9]After Reranking, new rank list for nodes: [1, 3, 5, 8, 0, 2, 4, 7, 6, 9]After Reranking, new rank list for nodes: [0, 5, 3, 7, 8, 4, 2, 1, 9, 6]After Reranking, new rank list for nodes: [0, 3, 6, 2, 4, 5, 8, 

In [16]:
import pandas as pd


def display_results(name, eval_results):
    """Display results from evaluate."""

    metric_dicts = []
    for eval_result in eval_results:
        metric_dict = eval_result.metric_vals_dict
        metric_dicts.append(metric_dict)

    full_df = pd.DataFrame(metric_dicts)

    columns = {
        "retrievers": [name],
        **{k: [full_df[k].mean()] for k in metrics},
    }

    metric_df = pd.DataFrame(columns)

    return metric_df

In [17]:
display_results("top-2 eval", eval_results)

,retrievers,hit_rate,mrr
0,top-2 eval,0.899261,0.756015


#### 3.2. Saving to evaluation metrics and results to json:

In [19]:
import json
from llama_index.core.evaluation.retrieval.base import RetrievalEvalResult

def custom_save_json(eval_results: list[RetrievalEvalResult], path: str = './qa_dataset.json'):
    '''
    Saving list of RetrievalEvalResult to JSON file
    '''
    data = []

    for result in eval_results:
        metric_dict = {
            metric_name: score.score
            for metric_name, score in result.metric_dict.items()
        }

        data.append({
            "query": result.query,
            "expected_ids": result.expected_ids,
            "expected_texts": result.expected_texts,
            "retrieved_ids": result.retrieved_ids,
            "retrieved_texts": result.retrieved_texts,
            "metric_dict": metric_dict,
        })

    with open(path, 'w', encoding="utf-8") as f:
        json.dump(data, f, indent=4, ensure_ascii=False)

custom_save_json(eval_results, './retrieval_eval.json')


In [ ]:
# Generating Dataset: đảm bảo dataset có tính chất giống với real-life (cách hỏi và câu hỏi bắt chước người dùng) 
# --> chỉnh prompt để tạo ra câu hỏi sát với thực tế
# cần xây dựng pipeline hoàn thiện hơn bao gồm document selection và context compression
# --> Đọc document để bổ sung vào pipeline
# cần nhiều hơn metrics để đánh giá
# --> Hỉu thêm những metrics đánh giá
# Khác: Indexing và chunking hiệu quả hơn nữa
# --> làm sao ????
